# Annotation agreement

In [19]:
import re
import pke
import nltk
import numpy as np
import pandas as pd
from nltk import stem
from nltk.corpus import stopwords
from collections import defaultdict
from typing import Generator, Callable

nltk.download('stopwords')
stops = stopwords.words('english')

stemmer = stem.PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/e154817e/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
""" Clean the string by removing non-alphanumeric characters and converting to lowercase """
def clean_string(string: str) -> str:
    return re.sub(r'\W+','', string).lower() 
    
""" Stem the string and converting to lowercase """
def stem_string(string:str) -> str:
    return stemmer.stem(string, to_lowercase=True)

""" Pre-process a list of words """
def list_word_process(lst:list) -> Generator:
    for kp in lst:
        for word in kp.split():
            clean_word = re.sub('[()]', '', word)
            stem = stem_string(clean_word)
            if stem not in stops:
                yield(stem)
                
""" Perform exact match between two lists """
def exact_match(lst1:list, lst2:list) -> float:
    if lst1 == lst2:
        return 1
    lst1 = list(map(stem_string, lst1))
    lst2 = list(map(stem_string, lst2))
    inter = list(set(lst1) & set(lst2))
    union = list(set(lst1) | set(lst2))
    return len(inter) / len(union) if len(union) > 0 else 0
    
""" Perform word exact match between two lists """
def word_exact_match(lst1:list, lst2:list) -> float:
    return exact_match(list_word_process(lst1), list_word_process(lst2))
    
""" Score two annotations based on a score function """
def score(annotator1_keyphrases:dict, annotator2_keyphrases:dict, func:Callable=exact_match) -> float:
    return sum([func(anot1, anot2) for anot1, anot2 in zip(annotator1_keyphrases, annotator2_keyphrases)])/ max(len(annotator1_keyphrases), len(annotator2_keyphrases))

""" Display results as a matrix """
def matrix_annotations(annotations_dict:dict, func:Callable=word_exact_match, cols:list=[]) -> pd.DataFrame:
    data = defaultdict(lambda : defaultdict(list))
    for k, annotations in annotations_dict.items():
        for col in cols:
            data[k][col]= score(annotations, annotations_dict[col], func=func)
    return pd.DataFrame(data).transpose()
    
""" Split keyphrases as singular expressions """
def split_queries_keyphrases(queries:list) -> list:
    queries = queries.replace(np.nan, "")
    return [[v.strip() for v in query.strip().split(",")] if query.strip().split(",") != [''] else [] for query in queries]

In [22]:
reading_lists = pd.read_csv("reading_lists.csv")
reading_lists = reading_lists.replace(np.nan, None)

queries_kps = {}
for annotator_i in [1,2,3, "bart"]:
    queries_kps[annotator_i] = split_queries_keyphrases(pd.read_csv(f"annotations/annotation_{annotator_i}.csv")["query_keywords"])

## Mean keyphrases queries amount

In [23]:
pd.DataFrame({
    "A1": np.mean([len(kws) for kws in queries_kps[1]]),
    "A2": np.mean([len(kws) for kws in queries_kps[2]]), 
    "A3": np.mean([len(kws) for kws in queries_kps[3]]), 
    "Bart-Large-KP20K": np.mean([len(kws) for kws in queries_kps["bart"]])
}, index=["Mean # of KP"]).transpose()

,Mean # of KP
A1,2.705882
A2,3.835294
A3,2.341176
Bart-Large-KP20K,4.447059


## Agreement of annotators and comparison to baselines

In [24]:
firstphrase_title_keyphrases = []

for title in reading_lists["title"]:
    extractor = pke.unsupervised.FirstPhrases()
    extractor.load_document(input=title, language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=3)
    firstphrase_title_keyphrases.append([kp_tuple[0] for kp_tuple in keyphrases])

In [25]:
topicrank_keyphrases = []

for title,abstract in zip(reading_lists["title"],reading_lists["abstract"]):
    extractor = pke.unsupervised.TopicRank()
    extractor.load_document(input=title+"\n "+(abstract or ""), language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=3)
    topicrank_keyphrases.append([kp_tuple[0] for kp_tuple in keyphrases])

In [26]:
np.mean([len(kws) for kws in firstphrase_title_keyphrases])

2.3529411764705883

In [28]:
np.mean([len(kws) for kws in topicrank_keyphrases])

3.0

In [29]:
columns = ["A1", "A2", "A3"]
annotations = {
    "A1":queries_kps[1],
    "A2":queries_kps[2], 
    "A3":queries_kps[3], 

    "FirstPhrase Title":firstphrase_title_keyphrases, 
    "TopicRank":topicrank_keyphrases, 
    "Bart-Large-KP20K":queries_kps["bart"],
}

In [30]:
df = matrix_annotations(annotations, func=word_exact_match, cols=columns); df
#print(df.to_latex(float_format="%.2f"))

,A1,A2,A3
A1,1.000000,0.305374,0.537675
A2,0.305374,1.000000,0.313214
A3,0.537675,0.313214,1.000000
FirstPhrase Title,0.403280,0.269857,0.404577
TopicRank,0.368888,0.230942,0.407870
Bart-Large-KP20K,0.412099,0.325121,0.380824


## Agreement of annotators on sentence queries

In [36]:
queries_sentences = {}
for annotator_i in [1,2,3]:
    annotator_sentences = [query.strip() for query in pd.read_csv(f"annotations/annotation_{annotator_i}.csv")["query_sentence"].replace(np.nan, "")]
    queries_sentences[annotator_i] = [[clean_string(stem_string(w)) for w in s.split() if w not in stops] for s in annotator_sentences]    

In [37]:
columns = ["A1", "A2", "A3"]
annotations_sentences = {
    "A1":queries_sentences[1],
    "A2":queries_sentences[2], 
    "A3":queries_sentences[3], 
}

In [38]:
df = matrix_annotations(annotations_sentences, func=word_exact_match, cols=columns); df
#print(df.to_latex(float_format="%.2f"))

,A1,A2,A3
A1,1.000000,0.411043,0.603139
A2,0.411043,1.000000,0.406821
A3,0.603139,0.406821,1.000000
